### Test Corr Models

In [1]:
import numpy as np
import pandas as pd
from copy import deepcopy
import yfinance as yf

import CorrelationsModels as CM
import ExpectedShortfall as ES
import Backtesting as BT
import InteractivePlots as IP

In [2]:
# Step 1: Download prices
tickers = ["^GSPC", "^STOXX50E", "^N225"]
data = yf.download(tickers, start="2020-01-01", end="2024-01-01")["Close"]
data = data.dropna()

# Step 2: Define fixed quantities
quantities = pd.Series([10, 20, 30], index=tickers)

# Step 3: Compute monetary positions
x_matrix = data.multiply(quantities, axis=1)


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  3 of 3 completed


In [3]:
x_matrix.head()

,^GSPC,^N225,^STOXX50E
Date,,,
2020-01-06,32462.800293,696145.781250,75050.400391
2020-01-07,32371.799316,707271.621094,75185.000000
2020-01-08,32530.500488,696142.792969,75451.201172
2020-01-09,32746.999512,712196.074219,75917.597656
2020-01-10,32653.500977,715517.109375,75790.400391


In [4]:
# Run MA Correlation VaR
x_ma = deepcopy(x_matrix)
result_ma, next_day_ma = CM.var_corr_ma(x_ma, window_size=20)

# Run RiskMetrics Correlation VaR
x_rm = deepcopy(x_matrix)
result_rm, next_day_rm = CM.var_corr_riskmetrics(x_rm)


In [5]:
print("== MA Correlation VaR ==")
print(result_ma[["Returns", "Volatility", "VaR", "VaR Monetary", "VaR Violation"]].head())
print("Next-day MA VaR (monetary):", round(next_day_ma, 2), "\n")

print("== RiskMetrics Correlation VaR ==")
print(result_rm[["Returns", "Volatility", "VaR", "VaR Monetary", "VaR Violation"]].head())
print("Next-day RM VaR (monetary):", round(next_day_rm, 2))


== MA Correlation VaR ==
             Returns   Volatility       VaR  VaR Monetary  VaR Violation
2020-02-05  0.010412  8309.968739  0.021492  17375.824364          False
2020-02-06  0.021416  8904.923516  0.023030  19017.940880          False
2020-02-07 -0.002046  8724.596788  0.022564  18594.699474          False
2020-02-10 -0.004994  7897.955967  0.020426  16748.678461          False
2020-02-12  0.008245  7971.821672  0.020617  17044.600062          False
Next-day MA VaR (monetary): 34289.64 

== RiskMetrics Correlation VaR ==
             Returns    Volatility       VaR  VaR Monetary  VaR Violation
Date                                                                     
2020-01-07  0.013927  10088.192216  0.024396  19878.405711          False
2020-01-08 -0.013091   9978.986466  0.024132  19404.915610          False
2020-01-09  0.020845  10739.575218  0.025971  21318.595329          False
2020-01-10  0.003782  10484.792053  0.025355  20891.446756          False
2020-01-14  0.006228

In [6]:
# Just call the function on either result:
fig_ma = IP.interactive_plot_var(result_ma)
fig_ma.show()

# Or for RiskMetrics:
fig_rm = IP.interactive_plot_var(result_rm)
fig_rm.show()

In [7]:
# Apply volatility-based ES to the MA VaR result
result_ma = ES.compute_expected_shortfall_volatility(result_ma, confidence_level=0.99)

# Same for RiskMetrics
result_rm = ES.compute_expected_shortfall_volatility(result_rm, confidence_level=0.99)

In [8]:
# Plot MA model results
fig_ma_es = IP.interactive_plot_es(result_ma)
fig_ma_es.show()

# Plot RiskMetrics model results
fig_rm_es = IP.interactive_plot_es(result_rm)
fig_rm_es.show()

In [9]:
violations_ma, rate_ma = BT.backtest_var(result_ma, confidence_level=0.99)

print("== MA Correlation VaR Backtest ==")
print(f"Total Violations: {violations_ma}")
print(f"Violation Rate: {rate_ma:.2f}%")

== MA Correlation VaR Backtest ==
Total Violations: 10
Violation Rate: 1.10%


In [10]:
violations_rm, rate_rm = BT.backtest_var(result_rm, confidence_level=0.99)

print("== RiskMetrics Correlation VaR Backtest ==")
print(f"Total Violations: {violations_rm}")
print(f"Violation Rate: {rate_rm:.2f}%")

== RiskMetrics Correlation VaR Backtest ==
Total Violations: 13
Violation Rate: 1.40%


In [11]:
fig_ma_vol = IP.interactive_plot_volatility(result_ma["Volatility"])
fig_rm_vol = IP.interactive_plot_volatility(result_rm["Volatility"])

fig_ma_vol.show()
fig_rm_vol.show()